In [1]:
%load_ext autoreload
%autoreload 2

from numpy.random import seed
seed(123)

In [2]:
from scripts.audio_feature_extraction_for_file import extract_audio_features_for_frames
from scripts.biosignal_data_extraction import get_biosignal_data_for_frames
from scripts.video_landmark_extraction import get_mediapipe_pose_estimation_for_frames, get_dlib_face_estimation_for_frames
from scripts.process_data import process_data_folder, check_audio_feature_length
from scripts.metadata_extraction import extract_metadata

2024-06-26 18:40:05.245467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/Suvi/miniforge3/envs/data_extract_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Save the lookup table to a CSV file -- only run once to get the file if it doesn't already exist 
# create_and_save_lookup_table('frequency_to_note.csv')

## 1. Testing

#### 1.1. Test audio feature extraction

In [3]:
# Test that feature extraction works as expected on test audio
TEST_FILE = 'media/test_audio.wav'
audio_features, cleaned_time = extract_audio_features_for_frames(TEST_FILE, source='mic', reference_audio=True, frame_duration_ms=30)

9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 428ms/step


In [4]:
audio_features

,mic_pitch,mic_note,mic_rms_energy,mic_spec_cent,mic_spec_spread,mic_spec_skew,mic_spec_kurt,mic_spec_slope,mic_spec_decr,mic_spec_rolloff,...,mic_mfcc_4,mic_mfcc_5,mic_mfcc_6,mic_mfcc_7,mic_mfcc_8,mic_mfcc_9,mic_mfcc_10,mic_mfcc_11,mic_mfcc_12,mic_mfcc_13
0,223.434784,A3,0.000360,5091.266602,0.000199,11.248163,165.800018,-0.000023,-6.796704e-07,399.697876,...,4.583538,13.187982,10.358666,-0.561259,-5.324896,-4.873797,-6.282567,-3.121312,4.438794,0.691595
1,225.978027,A3,0.003805,2677.899414,0.018409,11.721117,161.712204,-0.000185,-5.549279e-06,299.773407,...,15.360875,4.172635,-1.448455,-8.647213,-11.089440,-8.763836,-5.968566,-5.472165,-2.669916,-2.329993
2,220.492004,A3,0.008556,1270.129883,0.094255,12.438404,194.393448,-0.000421,-1.263218e-05,466.314209,...,6.560631,-9.154160,-2.909495,-0.738289,-7.055696,-13.285758,4.190375,-4.283062,1.877032,-5.371984
3,217.796326,A3,0.008928,1251.080811,0.101976,10.466989,129.784988,-0.000483,-1.449109e-05,433.006042,...,3.123775,-12.050467,4.027173,7.928208,-0.242536,-19.874636,7.789165,-7.561949,-5.149116,-3.780768
4,218.802826,A3,0.009930,1434.268188,0.125384,9.589105,106.888527,-0.000562,-1.688281e-05,866.012085,...,-1.814801,-11.623101,6.149551,6.418947,-3.282065,-15.764326,0.778008,-8.930828,-2.869854,-5.087080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,221.457047,A3,0.002967,2073.424072,0.011396,15.029715,263.597412,-0.000118,-3.552412e-06,233.157104,...,14.163931,4.501680,1.801187,0.317944,-3.753589,-4.025735,-5.004305,-7.006836,-3.944653,-3.762465
82,221.902328,A3,0.001967,3077.006104,0.005010,15.995955,303.774292,-0.000085,-2.563286e-06,233.157104,...,10.414039,3.120530,-0.501672,-2.172984,-5.263518,-9.741070,-10.443014,-9.269808,-7.636166,-3.145139
83,222.313446,A3,0.001276,3557.038574,0.002084,16.873587,338.649780,-0.000051,-1.541668e-06,233.157104,...,13.887827,6.871558,4.580100,-2.023361,-6.288260,-8.562696,-9.202233,-8.121056,-7.173820,-4.689435
84,221.693771,A3,0.000876,4149.004883,0.000979,16.741419,325.111633,-0.000034,-1.021123e-06,233.157104,...,10.029733,1.254643,2.842723,1.438787,-1.009661,-0.758479,-7.669601,-8.499184,-6.028996,-2.336360


In [5]:
mic_feature_length = check_audio_feature_length(audio_features, 'mic')
mic_feature_length

86

#### 1.2. Test video feature extraction

In [10]:
# testing getting mediapipe landmark pose per frame
TEST_PHONE_VIDEO_FILE_PATH = 'media/test_phone_video.mp4'
TEST_COMPUTER_VIDEO_FILE_PATH = 'media/test_computer_video.mp4'

In [7]:
phone_pose_landmarks = get_mediapipe_pose_estimation_for_frames(cleaned_time, TEST_PHONE_VIDEO_FILE_PATH, source='Phone')
if len(phone_pose_landmarks) != mic_feature_length:
    raise ValueError(f"Pose frame number must match audio feature frame length")

I0000 00:00:1719420127.589542 1883653 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 570 OpenGL Engine
Processing Frames:   0%|          | 0/67 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719420129.271934 1886589 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719420129.329641 1886590 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/Suvi/miniforge3/envs/data_extract_env/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing Frames: 100%|██████████| 67/67

Ignoring empty video frame.


In [8]:
phone_pose_landmarks

,Phone_pose_landmark_1_x,Phone_pose_landmark_1_y,Phone_pose_landmark_1_z,Phone_pose_landmark_2_x,Phone_pose_landmark_2_y,Phone_pose_landmark_2_z,Phone_pose_landmark_3_x,Phone_pose_landmark_3_y,Phone_pose_landmark_3_z,Phone_pose_landmark_4_x,...,Phone_pose_landmark_30_z,Phone_pose_landmark_31_x,Phone_pose_landmark_31_y,Phone_pose_landmark_31_z,Phone_pose_landmark_32_x,Phone_pose_landmark_32_y,Phone_pose_landmark_32_z,Phone_pose_landmark_33_x,Phone_pose_landmark_33_y,Phone_pose_landmark_33_z
0,0.458994,0.286738,-0.318805,0.462257,0.270094,-0.312873,0.466212,0.267767,-0.312844,0.469857,...,0.264428,0.483865,0.811781,0.248081,0.532783,0.869385,0.195743,0.473384,0.881753,0.180039
1,0.458998,0.286754,-0.281170,0.461943,0.270077,-0.277489,0.465551,0.267704,-0.277445,0.469162,...,0.219848,0.483981,0.816704,0.225352,0.533367,0.869654,0.158152,0.472502,0.882623,0.163137
2,0.459119,0.286994,-0.293268,0.461911,0.270082,-0.287307,0.465452,0.267694,-0.287290,0.469069,...,0.233091,0.484095,0.819044,0.231644,0.533540,0.868946,0.167774,0.472054,0.883592,0.165035
3,0.459419,0.287592,-0.298975,0.461916,0.270163,-0.294568,0.465454,0.267702,-0.294525,0.469071,...,0.237863,0.484202,0.820385,0.233770,0.533636,0.869123,0.171825,0.472022,0.884292,0.167334
4,0.459419,0.287592,-0.298975,0.461916,0.270163,-0.294568,0.465454,0.267702,-0.294525,0.469071,...,0.237863,0.484202,0.820385,0.233770,0.533636,0.869123,0.171825,0.472022,0.884292,0.167334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0.463259,0.288084,-0.311652,0.464959,0.270368,-0.306967,0.468630,0.267460,-0.306919,0.472278,...,0.233835,0.485105,0.818193,0.233740,0.534504,0.869986,0.171145,0.471783,0.884730,0.170321
82,0.463282,0.288019,-0.310026,0.465009,0.270310,-0.305311,0.468682,0.267405,-0.305266,0.472319,...,0.230632,0.485129,0.818200,0.231892,0.534554,0.870259,0.167430,0.471799,0.884703,0.168119
83,0.463365,0.287999,-0.302563,0.465105,0.270289,-0.297948,0.468772,0.267384,-0.297900,0.472389,...,0.223933,0.485109,0.817906,0.224291,0.534538,0.870264,0.162130,0.471795,0.884711,0.161789
84,0.463365,0.287999,-0.302563,0.465105,0.270289,-0.297948,0.468772,0.267384,-0.297900,0.472389,...,0.223933,0.485109,0.817906,0.224291,0.534538,0.870264,0.162130,0.471795,0.884711,0.161789


In [11]:
computer_pose_landmarks = get_mediapipe_pose_estimation_for_frames(cleaned_time, TEST_COMPUTER_VIDEO_FILE_PATH, source='Phone')
if len(computer_pose_landmarks) != mic_feature_length:
    raise ValueError(f"Pose frame number must match audio feature frame length")

I0000 00:00:1719420223.231873 1883653 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 570 OpenGL Engine
Processing Frames:   0%|          | 0/67 [00:00<?, ?it/s]W0000 00:00:1719420223.584955 1887457 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719420223.649985 1887457 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/Suvi/miniforge3/envs/data_extract_env/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing Frames: 100%|██████████| 67/67 [00:02<00:00, 23.49it/s]


Ignoring empty video frame.


In [12]:
computer_pose_landmarks

,Phone_pose_landmark_1_x,Phone_pose_landmark_1_y,Phone_pose_landmark_1_z,Phone_pose_landmark_2_x,Phone_pose_landmark_2_y,Phone_pose_landmark_2_z,Phone_pose_landmark_3_x,Phone_pose_landmark_3_y,Phone_pose_landmark_3_z,Phone_pose_landmark_4_x,...,Phone_pose_landmark_30_z,Phone_pose_landmark_31_x,Phone_pose_landmark_31_y,Phone_pose_landmark_31_z,Phone_pose_landmark_32_x,Phone_pose_landmark_32_y,Phone_pose_landmark_32_z,Phone_pose_landmark_33_x,Phone_pose_landmark_33_y,Phone_pose_landmark_33_z
0,0.514709,0.194062,-0.54944,0.517368,0.167349,-0.537648,0.521297,0.165255,-0.537658,0.525019,...,None,None,None,None,None,None,None,None,None,None
1,0.515306,0.193888,-0.526235,0.51772,0.167317,-0.512062,0.521725,0.165205,-0.512129,0.525423,...,None,None,None,None,None,None,None,None,None,None
2,0.515326,0.192896,-0.527737,0.517793,0.166809,-0.513971,0.521866,0.164678,-0.51403,0.525618,...,None,None,None,None,None,None,None,None,None,None
3,0.515321,0.187843,-0.514986,0.517756,0.162888,-0.50037,0.521842,0.160707,-0.500443,0.525609,...,None,None,None,None,None,None,None,None,None,None
4,0.515321,0.187843,-0.514986,0.517756,0.162888,-0.50037,0.521842,0.160707,-0.500443,0.525609,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0.520487,0.18254,-0.53414,0.521894,0.158981,-0.517659,0.525867,0.15703,-0.517711,0.5299,...,None,None,None,None,None,None,None,None,None,None
82,0.52045,0.18251,-0.531487,0.521844,0.15896,-0.514896,0.525823,0.157016,-0.514954,0.529847,...,None,None,None,None,None,None,None,None,None,None
83,0.520392,0.182536,-0.528509,0.521797,0.158974,-0.512021,0.525785,0.157036,-0.512081,0.529804,...,None,None,None,None,None,None,None,None,None,None
84,0.520392,0.182536,-0.528509,0.521797,0.158974,-0.512021,0.525785,0.157036,-0.512081,0.529804,...,None,None,None,None,None,None,None,None,None,None


In [13]:
face_landmarks = get_dlib_face_estimation_for_frames(cleaned_time, TEST_PHONE_VIDEO_FILE_PATH, source='Phone')
if len(face_landmarks) != mic_feature_length:
    raise ValueError(f"Pose frame number must match audio feature frame length")

Processing Frames: 100%|██████████| 67/67 [00:13<00:00,  4.85it/s]
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use locals() to access x, y, z by string
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use locals

In [14]:
face_landmarks

,Phone_face_landmark_1_x,Phone_face_landmark_1_y,Phone_face_landmark_1_z,Phone_face_landmark_2_x,Phone_face_landmark_2_y,Phone_face_landmark_2_z,Phone_face_landmark_3_x,Phone_face_landmark_3_y,Phone_face_landmark_3_z,Phone_face_landmark_4_x,...,Phone_face_landmark_65_z,Phone_face_landmark_66_x,Phone_face_landmark_66_y,Phone_face_landmark_66_z,Phone_face_landmark_67_x,Phone_face_landmark_67_y,Phone_face_landmark_67_z,Phone_face_landmark_68_x,Phone_face_landmark_68_y,Phone_face_landmark_68_z
0,851.0,308.0,None,854.0,318.0,None,858.0,326.0,None,864.0,...,None,894.0,332.0,None,891.0,334.0,None,888.0,335.0,None
1,851.0,309.0,None,854.0,318.0,None,859.0,326.0,None,864.0,...,None,893.0,333.0,None,890.0,335.0,None,887.0,335.0,None
2,851.0,308.0,None,854.0,318.0,None,859.0,326.0,None,864.0,...,None,894.0,336.0,None,891.0,338.0,None,887.0,338.0,None
3,852.0,309.0,None,855.0,318.0,None,860.0,327.0,None,865.0,...,None,894.0,336.0,None,891.0,338.0,None,888.0,338.0,None
4,852.0,309.0,None,855.0,318.0,None,860.0,327.0,None,865.0,...,None,894.0,336.0,None,891.0,338.0,None,888.0,338.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,853.0,308.0,None,857.0,318.0,None,863.0,326.0,None,868.0,...,None,899.0,330.0,None,896.0,331.0,None,893.0,332.0,None
82,855.0,308.0,None,858.0,318.0,None,863.0,326.0,None,868.0,...,None,899.0,330.0,None,896.0,331.0,None,892.0,332.0,None
83,855.0,307.0,None,859.0,317.0,None,863.0,325.0,None,869.0,...,None,899.0,330.0,None,896.0,331.0,None,893.0,332.0,None
84,855.0,307.0,None,859.0,317.0,None,863.0,325.0,None,869.0,...,None,899.0,330.0,None,896.0,331.0,None,893.0,332.0,None


#### 1.3. Test biosignal extraction

In [15]:
TEST_BIO_FILE_PATH = 'media/test_biosignal_data.json'

biosignal_data_df = get_biosignal_data_for_frames(cleaned_time, TEST_BIO_FILE_PATH, frame_duration_ms=30)
if len(biosignal_data_df) != mic_feature_length:
        raise ValueError("Biosignal data frame number must match audio feature frame length")

In [16]:
biosignal_data_df

,RESPIRATION_1,EMG_1,EEG_1,EEG_2
0,28148,33039,32962,31766
1,25552,32660,34285,40372
2,27248,32532,24638,37706
3,27248,32532,24638,37706
4,27248,32532,24638,37706
...,...,...,...,...
81,27248,32532,24638,37706
82,27248,32532,24638,37706
83,27248,32532,24638,37706
84,27248,32532,24638,37706


#### 1.4. Test metadata extraction

In [ ]:
test_meta_file = "path_to_test_file_in_clean_data_dir"
meta_data = extract_metadata(test_meta_file)

In [130]:
meta_data

,participant_number,sex,age,experience_level,phonation,recording_condition,phrase,clip_number
0,14,F,24,intermediate,breathy,arched_back,glissando,1


#### 1.5. Test folder processing

In [18]:
# test feature extraction from file system
DIR_IN = '/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS'
CSV_TEST = 'data/out_test.csv'
LOG_TEST = 'data/df_processed_folders.log'

process_data_folder(DIR_IN, CSV_TEST, LOG_TEST, num_folders_to_process=1, frame_duration_ms=30)

Processing folders:   0%|          | 0/1 [00:00<?, ?it/s]

18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 440ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 415ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 416ms/step


I0000 00:00:1719420413.495234 1883653 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 570 OpenGL Engine
W0000 00:00:1719420413.867097 1889305 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719420413.909233 1889305 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/Suvi/miniforge3/envs/data_extract_env/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing Frames: 100%|██████████| 144/144 [00:04<00:00, 30.97it/s]


Ignoring empty video frame.


Processing Frames: 100%|██████████| 144/144 [00:28<00:00,  5.06it/s]
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use locals() to access x, y, z by string
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use loca

Ignoring empty video frame.


Processing Frames: 100%|██████████| 144/144 [00:31<00:00,  4.64it/s]
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use locals() to access x, y, z by string
/Users/Suvi/Documents/UPF/THESIS/thesis_code/feature_extraction/scripts/video_landmark_extraction.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  reshaped_df.loc[idx, new_col_name] = locals()[coord]  # Use loca

Processed 1 folders. Stopping.


## 2. Running on the whole folder system

In [ ]:
# FINAL RUNNING PROCESSING OF ALL DATA

DIR_IN = '/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS'
CSV_OUT = "/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS/voice_data_thesis.csv"
LOG_FILE = "/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS/processed_folders.log"

process_data_folder(DIR_IN, CSV_OUT, LOG_FILE, num_folders_to_process=0, frame_duration_ms=30)


## 3. Post process CSV -- DO NOT RUN! Code refactored.

In [ ]:
# change CSV file to have the correct format for the data -- audio
# from scripts.csv_post_process import fix_csv_format

# CSV_NEW = "/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS/voice_data_thesis_fixed.csv"
# fix_csv_format(CSV_OUT, CSV_NEW)